In [1]:
import pandas as pd
import datetime
import numpy as np

In [2]:
exchanges = {
    'binance':
        {
            'bitcoin':'BTC',
            'ethereum':'ETH',
            'litecoin':'LTC',
            'solana':'SOL',
            'ripple':'XRP',
            'dollar':'USDT',
        },
    'coinbase':
        {
            'bitcoin':'BTC',
            'ethereum':'ETH',
            'litecoin':'LTC',
            'solana':'SOL',
            #'ripple':'xrp',
            'dollar':'USD',
        }, 
    'ftx':
        {
            'bitcoin':'BTC',
            'ethereum':'ETH',
            'litecoin':'LTC',
            'solana':'SOL',
            'ripple':'XRP',
            'dollar':'USD',
        }, 
    'bitmex':
        {
            'bitcoin':'XBT',
            'ethereum':'ETH',
            'litecoin':'LTC',
            'solana':'SOL',
            'ripple':'XRP',
            'dollar':'USD',
        },
    'kraken':
        {
            'bitcoin':'XBT',
            'ethereum':'ETH',
            'litecoin':'LTC',
            'solana':'SOL',
            'ripple':'XRP',
            'dollar':'USD',
        },
    }


In [3]:
date = datetime.datetime(2022, 2, 28)
date_str = date.strftime('%Y-%m-%d')

In [4]:
metrics = [
    'row_count', 
    'bid_ask_spread', 
    'avg_diff_ask_0_1', 
    'avg_diff_ask_1_2', 
    'avg_diff_ask_2_3',
    'avg_diff_ask_3_4', 
    'avg_diff_bid_0_1', 
    'avg_diff_bid_1_2', 
    'avg_diff_bid_2_3',
    'avg_diff_bid_3_4', 
    'avg_vol_ask_0',
    'avg_vol_ask_1',
    'avg_vol_ask_2',
    'avg_vol_ask_3',
    'avg_vol_ask_4',
    'avg_vol_bid_0',
    'avg_vol_bid_1',
    'avg_vol_bid_2',
    'avg_vol_bid_3',
    'avg_vol_bid_4',
]

In [11]:
# summary_table = [['crypto'], ['metric'], ['binance'],['coinbase'],['ftx'],['bitmex'],['kraken']]
nb_metrics = len(metrics)
summary_table = np.zeros(( nb_metrics*(len(exchanges['binance'])-1), len(exchanges) ))
for a,asset in enumerate(exchanges['binance']):
    print(asset)
    idx = (a)*nb_metrics
    for e,exch in enumerate(exchanges):
        print(exch)
        try:
            if exch == 'coinbase' and asset=='solana':
                path = 'tardis_raw.nosync/'+exch+'/'+exch+'_book_snapshot_5_'+date_str+'_'+exchanges[exch][asset]+'USDT'+'.csv.gz'
            else:
                path = 'tardis_raw.nosync/'+exch+'/'+exch+'_book_snapshot_5_'+date_str+'_'+exchanges[exch][asset]+exchanges[exch]['dollar']+'.csv.gz'
            ob = pd.read_csv(path, compression='gzip')
            # row_count
            rows = ob.shape[0]
            summary_table[idx+0,e] = rows
            # bid_ask_spread
            # summary_table[idx+1,e] = (ob['asks[0].price'] - ob['bids[0].price']).mean()
            summary_table[idx+1,e] = 1/rows * (ob['asks[0].price'].sum() - ob['bids[0].price'].sum())
            for k in range(5):
                try:
                    # avg_vol_ask_k
                    summary_table[idx+10+k,e] = 1/rows * ob['asks['+str(k)+'].amount'].sum()
                    # avg_vol_bid_k
                    summary_table[idx+15+k,e] = 1/rows * ob['bids['+str(k)+'].amount'].sum()
                    # avg_diff_ask_k_k+1
                    summary_table[idx+2+k,e] = 1/rows * (ob['asks['+str(k+1)+'].price'].sum() - ob['asks['+str(k)+'].price'].sum())
                    # avg_diff_bid_k_k+1
                    summary_table[idx+6+k,e] = 1/rows * (ob['bids['+str(k)+'].price'].sum() - ob['bids['+str(k+1)+'].price'].sum())
                except Exception as e:
                    print(e)
                    pass
        except Exception as e:
            print(e)
            pass

bitcoin
binance
'asks[5].price'
coinbase
'asks[5].price'
ftx
'asks[5].price'
bitmex
'asks[5].price'
kraken
'asks[5].price'
ethereum
binance
'asks[5].price'
coinbase
'asks[5].price'
ftx
'asks[5].price'
bitmex
'asks[5].price'
kraken
'asks[5].price'
litecoin
binance
'asks[5].price'
coinbase
'asks[5].price'
ftx
'asks[5].price'
bitmex
'asks[5].price'
kraken
'asks[5].price'
solana
binance
'asks[5].price'
coinbase
'asks[5].price'
ftx
'asks[5].price'
bitmex
'asks[5].price'
kraken
'asks[5].price'
ripple
binance
'asks[5].price'
coinbase
'ripple'
ftx
'asks[5].price'
bitmex
'asks[5].price'
kraken
'asks[5].price'
dollar
binance
[Errno 2] No such file or directory: 'tardis_raw.nosync/binance/binance_book_snapshot_5_2022-02-28_USDTUSDT.csv.gz'
coinbase
[Errno 2] No such file or directory: 'tardis_raw.nosync/coinbase/coinbase_book_snapshot_5_2022-02-28_USDUSD.csv.gz'
ftx
[Errno 2] No such file or directory: 'tardis_raw.nosync/ftx/ftx_book_snapshot_5_2022-02-28_USDUSD.csv.gz'
bitmex
[Errno 2] No such f

In [27]:
cryptos = [[crypto for i in range(len(metrics))] for crypto in exchanges['binance']]
#remove dollar
cryptos.pop(len(cryptos)-1)
metric = [metrics for i in range(len(exchanges['binance'])-1)]

In [30]:
#df_summary_table = pd.DataFrame(list(zip(cryptos, metric, summary_table), columns=['asset', 'metric', 'binance', 'coinbase', 'ftx', 'bitmex', 'kraken'])
df_summary_table = pd.DataFrame(summary_table, columns=['binance', 'coinbase', 'ftx', 'bitmex', 'kraken'])
df_summary_table.insert(loc=0, column='Asset', value=np.array(cryptos).flatten())
df_summary_table.insert(loc=1, column='Metrics', value=np.array(metric).flatten())


In [ ]:
df_summary_table = df_summary_table.reset_index(drop=True)
df_summary_table.to_csv('order_books_stats.csv')